In [ ]:
import argparse
from collections import OrderedDict
import datetime
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pylab as plt
from numbers import Number
import numpy as np
import pandas as pd
import gc
pd.options.display.max_rows = 1500
pd.options.display.max_columns = 200
pd.options.display.width = 1000
pd.set_option('max_colwidth', 400)
import pdb
import pickle
import pprint as pp
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from deepsnap.batch import Batch as deepsnap_Batch

import sys, os
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', '..'))
from le_pde.argparser import arg_parse
from le_pde.datasets.load_dataset import load_data
from le_pde.models import load_model
from le_pde.pytorch_net.util import groupby_add_keys, filter_df, get_unique_keys_df, Attr_Dict, Printer, get_num_params, get_machine_name, pload, pdump, to_np_array, get_pdict, reshape_weight_to_matrix, ddeepcopy as deepcopy, plot_vectors, record_data, filter_filename, Early_Stopping, str2bool, get_filename_short, print_banner, plot_matrices, get_num_params, init_args, filter_kwargs, to_string, COLOR_LIST
from le_pde.utils import update_legacy_default_hyperparam, EXP_PATH, deepsnap_to_pyg, LpLoss, to_cpu, to_tuple_shape, parse_multi_step, loss_op, get_device, get_data_next_step
from utils import compute_pressForce
#from le_pde.utils import deepsnap_to_pyg, LpLoss, to_cpu, to_tuple_shape, parse_multi_step, loss_op, get_device, get_data_next_step

device = torch.device("cuda:0")
p = Printer()

## 1. Functions:

In [ ]:
def plot_learning_curve(data_record):
    x_axis = np.arange(len(data_record["train_loss"]))
    plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    plt.plot(x_axis, data_record["train_loss"], label="train")
    plt.plot(x_axis, data_record["val_loss"], label="val")
    plt.plot(x_axis, data_record["test_loss"], label="test")
    plt.legend()
    plt.subplot(1,2,2)
    plt.semilogy(x_axis, data_record["train_loss"], label="train")
    plt.semilogy(x_axis, data_record["val_loss"], label="val")
    plt.semilogy(x_axis, data_record["test_loss"], label="test")
    plt.legend()
    plt.show()

## 2. Load Data:

In [ ]:
EXP_PATH = "./results/"

isplot = True
all_hash = [
    # "0LVoHLHQ_ampere4",
    # "zDOCitP9_ampere4",
    # "6en0gt6G_turing1",
    # "zHQu3EKe_turing2",
    # "2okNCadZ_turing3",
    # "I6EepBQI_turing3",
    # "clnAWVnz_hyperturing1",
    # "YDHgg+il_turing3",
    "HD2hmsb+_turing3",
]
hash_str = all_hash[0]
dirname = EXP_PATH + "naca_ellipse_2023-06-02/"
filename = filter_filename(dirname, include=hash_str)
if len(filename) == 0:
    raise

try:
    data_record = pload(dirname + filename[0])
except Exception as e:
    print(f"error {e}")
    # continue
    raise
if isplot:
    plot_learning_curve(data_record)
args = init_args(update_legacy_default_hyperparam(data_record["args"]))
args.filename = filename
# model = load_model(data_record["best_model_dict"], device=device)
model = load_model(data_record["model_dict"][-1], device=device)
model.eval()
p.print(filename, banner_size=100)

# Load test dataset:
args_test = deepcopy(args)
if args.temporal_bundle_steps == 1:
    if args.dataset in ["fno", "fno-2", "fno-3"]:
        args_test.multi_step = "20"
    elif args.dataset in ["fno-1"]:
        args_test.multi_step = "40"
    elif args.dataset in ["fno-4"]:
        args_test.multi_step = "10"
    elif args.dataset in ["naca_ellipse_lepde"]:
        args_test.multi_step = "1"
        args_test.latent_multi_step="1"
    else:
        raise
else:
    pass
args_test.batch_size = 1
args_test.is_test_only=True

(dataset_train_val, dataset_test), (train_loader, val_loader, test_loader) = load_data(args_test)
test_loader = DataLoader(dataset_test, num_workers=0, collate_fn=deepsnap_Batch.collate(),
                         batch_size=1, shuffle=False, drop_last=False)

In [ ]:
i=0
for data in test_loader:
    # if i == 1900:
    # if i == 2500:
    if i == 1200:
    # if i == 0:
    # if i == 400:
        break
    i+=1

    if i%100 == 0:
        fig, ax = plt.subplots(figsize=(4,4), ncols=1)
        bd = (((data.param["n0"].detach().cpu().numpy()/2) + 0.5) * 62) + 0
        mappable0 = ax.plot(bd[0,0::2], bd[0,1::2])
        ax.set_xlim(0, 62)
        ax.set_ylim(0, 62)
        plt.show()

In [ ]:
data.to(device)        
testdata = data.clone()

In [ ]:
# m = 64; n = 64
# maxnum = 100

# def discretize_boundary(boundary):
#     # import pdb
#     # pdb.set_trace()
#     assert boundary.shape[1] == 2
#     num_bound = boundary.shape[0]
#     device = boundary.device
#     p_5 = torch.tensor([0.5], device=device).repeat(num_bound)
#     x = torch.minimum(torch.maximum(boundary[:, 0], p_5), torch.tensor([n-1.5], device=device).repeat(num_bound))
#     x_inds = torch.minimum(x.type(torch.int32), torch.tensor([n-2], device=device).repeat(num_bound))
#     # fs = x - x_inds

#     y = torch.minimum(torch.maximum(boundary[:, 1], p_5), torch.tensor([m-1.5], device=device).repeat(num_bound))
#     y_inds = torch.minimum(y.type(torch.int32), torch.tensor([m-2], device=device).repeat(num_bound))
#     # ft = y - y_inds
#     return x_inds, y_inds

# def find_orthogonal_line(A, B, C, x0, y0):
#     m1 = torch.empty((C.shape[0],), device=C.device)
#     m1[B==0] = float('inf')
#     m1[B!=0] = (-A/B)[B!=0]

#     m2 = torch.empty((C.shape[0],), device=C.device)
#     m2[m1==float('inf')] = 0
#     m2[m1!=float('inf')] = (-1 / m1)[m1!=float('inf')]

#     b2 = y0 - m2 * x0  # The y-intercept of L2.

#     # Return the coefficients A, B, C of the line L2 (Ax + By - C = 0)
#     return m2, -1, b2

# def edge_cells(polygon):
#     num_vertices = len(polygon)
#     edges = []
#     for i in range(num_vertices):
#         v1 = polygon[i]
#         v2 = polygon[(i + 1) % num_vertices]
#         edge = sorted([v1, v2], key=lambda x: x[1])
#         edges.append(edge)
#     return edges

# def find_cells_inside_curve(polygon, grid_shape):
#     def horizontal_intersection(x1, y1, x2, y2, y):
#         return x1 + (y - y1) * (x2 - x1) / (y2 - y1)
#     edges = edge_cells(polygon)
#     grid = np.zeros(grid_shape, dtype=np.uint8)
#     height, width = grid.shape
    
#     for y in range(height):
#         intersections = []
#         for edge in edges:
#             y1, y2 = edge[0][1], edge[1][1]
#             if y1 < y <= y2:
#                 x = horizontal_intersection(*edge[0], *edge[1], y)
#                 intersections.append(x)
#         intersections.sort()
#         for i in range(0, len(intersections), 2):
#             x_start, x_end = int(np.ceil(intersections[i])), int(np.floor(intersections[i + 1]))
#             grid[y, x_start : x_end + 1] = 1

#     return grid

# def update_static_masks(torch_con_boundary):
#     # import pdb
#     # pdb.set_trace()
#     x_inds, y_inds = discretize_boundary(torch_con_boundary)
#     pointy_hash = maxnum*x_inds[20] + y_inds[20]

#     indices = torch.stack((maxnum*x_inds,y_inds), 0)
#     sum_indices = indices.sum(0)
#     ind_unique = torch.unique(sum_indices, sorted=True) #, return_inverse=True)
#     x_idx = (torch.cat([(sum_indices==ind_u).nonzero()[0] for ind_u in ind_unique])).sort()[0]
#     # print(sum_indices[x_idx])
#     # print(x_idx)
#     repeat_sum_indices = torch.tile(sum_indices, (ind_unique.shape[0],1))
#     repeat_ind_unique = torch.tile(sum_indices[x_idx].reshape(ind_unique.shape[0], 1), (1, sum_indices.shape[0]))
#     org_mask = (repeat_ind_unique == repeat_sum_indices)
#     fatted_mask = torch.roll(org_mask, 1, 1) + torch.roll(org_mask, -1, 1)

#     relvecs = []
#     base_pts = []
#     base_nums = []
#     for bdpt in range(sum_indices[x_idx].shape[0]):
#         # i = 1
#         if pointy_hash == sum_indices[x_idx][bdpt]:
#             base_pt = torch.stack([x_inds[org_mask[bdpt]][0], y_inds[org_mask[bdpt]][0]]) + 0.5
#             base_pts.append(base_pt)
#             relvec = torch_con_boundary[20] - base_pt
#             relvecs.append(relvec)
#             # base_nums.append(i)
#         elif torch.sum(org_mask[bdpt]) >= 4:
#             base_pt = torch.stack([x_inds[org_mask[bdpt]][0], y_inds[org_mask[bdpt]][0]]) + 0.5
#             base_pts.append(base_pt)
#             relvec = torch_con_boundary[org_mask[bdpt]] - base_pt.repeat(torch_con_boundary[org_mask[bdpt]].shape[0], 1)
#             ind = torch.argmin(torch.norm(relvec, dim=1))
#             relvecs.append(relvec[ind])
#             # base_nums.append(i)
#         elif torch.sum(fatted_mask[bdpt] * torch.logical_not(org_mask[bdpt])) > 2:
#             base_pt = torch.stack([x_inds[org_mask[bdpt]][0], y_inds[org_mask[bdpt]][0]]) + 0.5
#             # base_pts.append(base_pt)
#             relvec = torch_con_boundary[org_mask[bdpt]] - base_pt.repeat(torch_con_boundary[org_mask[bdpt]].shape[0], 1)
#             # print("disjoint: ", relvec)
#             if len(relvec.shape) == 2:
#                 import pdb
#                 # pdb.set_trace()
#                 # for i in range(xbound[org_mask[bdpt]].shape[0]):
#                     # if 2 * i + 2 == bd_points.shape[0]:
#                 # print("mask:", org_mask[bdpt])
#                 # print(xbound[org_mask[bdpt]])
#                 # plt.plot(xbound[org_mask[bdpt]][:, 0].numpy(), xbound[org_mask[bdpt]][:, 1].numpy())
#                 #     # else:
#                 #     #    plt.plot(bd_points[2*i:2*i+2, 0].numpy(), bd_points[2*i:2*i+2, 1].numpy())
#                 # plt.scatter(base_pt[0].numpy(), base_pt[1].numpy())
#                 # plt.show()
#                 # for i in range(relvec.shape[0]):
#                 #     row_relvec = relvec[i]
#                 #     relvecs.append(row_relvec)
#                 #     base_pts.append(base_pt)
#                 #     i += 1
#                 relvecs.append(relvec[-1])
#                 base_pts.append(base_pt)
#                 # base_nums.append(i)
#             else:
#                 relvecs.append(relvec)
#                 base_pts.append(base_pt)
#                 # base_nums.append(i)
#         elif torch.sum(org_mask[bdpt]) == 1:
#             base_pt = torch.stack([x_inds[org_mask[bdpt]][0], y_inds[org_mask[bdpt]][0]]) + 0.5
#             base_pts.append(base_pt)
#             relvec = torch_con_boundary[org_mask[bdpt]] - base_pt
#             # print("rel: ", relvec[0])
#             relvecs.append(relvec[0])
#             # base_nums.append(i)
#         else:    
#             if fatted_mask[bdpt][0] and fatted_mask[bdpt][-1]:
#                 rollnum = 1
#                 for _ in range(0, 100):
#                     temprole = torch.roll(fatted_mask[bdpt], rollnum, 0)
#                     if temprole[0] and temprole[-1]:
#                         rollnum += 1    
#                     else:
#                         break
#                 #import pdb
#                 #pdb.set_trace()
#                 x_pts = torch.roll(torch_con_boundary[fatted_mask[bdpt]], rollnum, 0)            
#             else:
#                 x_pts = torch_con_boundary[fatted_mask[bdpt]]

#             bd_points = torch.cat([x_pts[0:1], x_pts[1:-1].repeat(1, 2).reshape(-1,2), x_pts[-1:]], dim=0)
#             dire_vec = bd_points[0::2] - bd_points[1::2]
#             const = bd_points[0::2, 1] - bd_points[0::2, 0] * dire_vec[:,1]/dire_vec[:,0]

#             base_pt = torch.stack([x_inds[org_mask[bdpt]][0], y_inds[org_mask[bdpt]][0]]) + 0.5
#             base_pts.append(base_pt)
#             # base_nums.append(i)
#             base_points = base_pt.repeat(const.shape[0], 1)
#             slope = dire_vec[:,1]/dire_vec[:,0]
#             # import pdb
#             # pdb.set_trace()
#             ax, by, con = find_orthogonal_line(slope, -torch.ones((const.shape[0],), device=torch_con_boundary.device), const, base_points[:,0], base_points[:,1])

#             # for i in range(int(bd_points.shape[0]/2)):
#             #     if 2 * i + 2 == bd_points.shape[0]:
#             #         plt.plot(bd_points[2*i:, 0].numpy(), bd_points[2*i:, 1].numpy())
#             #     else:
#             #         plt.plot(bd_points[2*i:2*i+2, 0].numpy(), bd_points[2*i:2*i+2, 1].numpy())
#             # plt.scatter(base_pt[0].numpy(), base_pt[1].numpy())
#             # plt.show()

#             al = -ax/by
#             bl = con
#             cl = dire_vec[:,1]/dire_vec[:,0]
#             dl = const

#             # print(org_mask[bdpt,:])
#             intersection = torch.stack([(dl - bl)/(al - cl), (al*dl - bl*cl)/(al - cl)]).t()
#             # print(intersection)
#             # print(base_pt)

#             relvec = intersection - torch.tile(base_pt, (intersection.shape[0], 1))
#             #print(relvec.sum(0)/relvec.shape[0])
#             relvecs.append(relvec.sum(0)/relvec.shape[0])

#     ### Check number of offset vectors is same as that of boundary cells of solid
#     # print(len(base_pts), sum_indices[x_idx].shape[0])
#     assert len(base_pts) == sum_indices[x_idx].shape[0]
    
#     # import pdb
#     # pdb.set_trace()    
    
#     bd_offset = torch.stack(relvecs)
#     offset_grid_bound = torch.zeros((62, 62, 2), device=torch_con_boundary.device)
#     offset_grid_bound[x_inds, y_inds] = torch.tensor([1, 1], dtype=torch.float32, device=torch_con_boundary.device)
#     offset_grid_bound = offset_grid_bound.transpose(1,0)

#     # offset_grid = find_cells_inside_curve(torch.stack((x_inds, y_inds), -1).tolist(), grid_bound.shape)
#     offset_grid = find_cells_inside_curve(torch.stack((x_inds, y_inds), -1).detach().cpu().tolist(), (62, 62))
#     # fig, ax = plt.subplots(figsize=(4,4), ncols=1)
#     # ax.imshow(offset_grid, cmap='viridis',
#     #          aspect='auto',
#     #          origin='lower')
#     # plt.show()
#     inner_solid_mask = np.copy(offset_grid)
#     offset_grid = offset_grid.reshape(62, 62, 1)
#     offset_grid = np.concatenate([offset_grid, offset_grid], -1)

#     offset_union = offset_grid_bound + torch.tensor(offset_grid, device=torch_con_boundary.device)
#     offset_union[(offset_union.sum(-1) > 2),:] = torch.tensor([1, 1], dtype=torch.float32, device=torch_con_boundary.device)
#     offset_union.index_put_((y_inds[x_idx], x_inds[x_idx]), bd_offset)    

#     # np_offset_union = offset_union.detach().cpu().numpy()

#     # fig, ax = plt.subplots(figsize=(4,4), ncols=1)
#     # ax.imshow(np_offset_union[...,1], cmap='viridis',
#     #          aspect='auto',
#     #          origin='lower')
#     # plt.show()

#     # gtmask = data.node_feature["n0"][:,0,2].reshape(62,62).detach().cpu().numpy()
#     # fig, ax = plt.subplots(figsize=(4,4), ncols=1)
#     # ax.imshow(gtmask, cmap='viridis',
#     #          aspect='auto',
#     #          origin='lower')
#     # plt.show()

#     # print((data.node_feature["n0"][:,-1,1].reshape(62,62).detach().cpu().numpy() - np_offset_union[...,0]).sum())
#     # print((data.node_feature["n0"][:,-1,2].reshape(62,62).detach().cpu().numpy() - np_offset_union[...,1]).sum())
    
#     # updated_offset_mask = np_offset_union

#     grid_bound = torch.zeros((62, 62), device=torch_con_boundary.device)
#     grid_bound[x_inds, y_inds] = 1
#     # union = grid_bound.transpose(1,0).detach().cpu().numpy() + inner_solid_mask
#     union = grid_bound.transpose(1,0) + torch.tensor(inner_solid_mask, device=torch_con_boundary.device)
#     union[union == 2] = 1

#     # fig, ax = plt.subplots(figsize=(8,4), ncols=2)
#     # ax[0].imshow(union.flatten().reshape(62,62), cmap='viridis',
#     #          aspect='auto',
#     #          origin='lower')
#     # ax[1].imshow(data.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy(), cmap='viridis',
#     #          aspect='auto',
#     #          origin='lower')
#     # plt.show()

#     # print((data.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy() - union).sum())
#     updated_solid_mask = union
    
#     return updated_solid_mask, offset_union #updated_offset_mask

# def update_bdfeature(reconstructed_boundary):
#     upd_solid_mask, upd_solid_offset = update_static_masks(reconstructed_boundary)
#     torch_batch_mask = torch.where(upd_solid_mask==1, False, True).clone().flatten()
#     upd_solid_mask = upd_solid_mask[...,None]
#     static_feature = torch.cat((upd_solid_mask, upd_solid_offset), -1)
#     multi_static_feat = torch.stack([static_feature for _ in range(4)], -2).reshape(-1,4,3)
#     return multi_static_feat, torch_batch_mask

# def update_data(reconstructed_bound, optimdata, original_data, constant_var, opt_var):
#     mul_static_feat, tor_batch_mask = update_bdfeature(reconstructed_bound) 

#     a = deepsnap_Batch
#     batch, _ = a._init_batch_fields(optimdata.keys, [])
#     batch.batch = optimdata.batch.clone()
#     batch.compute_func = optimdata.compute_func
#     batch.directed = optimdata.directed.detach().clone()
#     batch.dyn_dims = optimdata.dyn_dims
#     batch.edge_attr = optimdata.edge_attr
#     batch.edge_index = {('n0','0','n0'): optimdata.edge_index[('n0','0','n0')].detach().clone()}
#     batch.edge_label_index = {('n0','0','n0'): optimdata.edge_label_index[('n0','0','n0')].detach().clone()}
#     batch.grid_keys = optimdata.grid_keys
#     batch.mask = {"n0": tor_batch_mask.detach()}
#     batch.node_feature = {"n0": torch.cat((mul_static_feat, original_data.node_feature["n0"][...,3:].detach()), -1)}
#     batch.node_label = {"n0": optimdata.node_label["n0"].detach().clone()}
#     batch.node_label_index = {"n0": optimdata.node_label_index["n0"].detach().clone()}
#     batch.node_pos = {"n0": optimdata.node_pos["n0"].detach().clone()}
#     batch.original_shape = optimdata.original_shape
#     batch.param = {"n0": torch.cat((constant_var, opt_var), 0).transpose(1,0).flatten()[None,:]}
#     batch.params = optimdata.params
#     batch.part_keys = optimdata.part_keys
#     batch.task = optimdata.task    
#     optimdata = batch    
    
#     return optimdata

## 3. inverse optimization with LEPDE

In [ ]:
testdata = data.clone()
orgdata = data.clone()
#print(testdata.param["n0"])
const_variable = testdata.param["n0"][:,0::2].detach().clone()
#const_variable.requires_grad=True
opt_variable = testdata.param["n0"][:,1::2].detach().clone()
opt_variable.requires_grad=True
#print(const_variable, opt_variable)
testdata.param["n0"] = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:]
#print(testdata.param["n0"])
# optimizer = torch.optim.Adam([opt_variable, const_variable], lr=0.01)
# best
# optimizer = torch.optim.Adam([opt_variable], lr=0.000005)
optimizer = torch.optim.Adam([opt_variable], lr=0.0001)

prerollout = 8
one_period = 4
vis_prerollout = False

# testdata = data.clone()
# original_optvar = testdata.param["n0"][:,1::2].detach().clone()

In [ ]:
normalization_filename = os.path.join("./dataset/naca_ellipse/training_trajectories/", "normalization_max_min.p")
normdict = pickle.load(open(normalization_filename, "rb"))
x_max = normdict["x_max"]
x_min = normdict["x_min"]
y_max = normdict["y_max"]
y_min = normdict["y_min"]
p_max = normdict["p_max"]
p_min = normdict["p_min"]
p_max = p_max.to(device)
p_min = p_min.to(device)

In [ ]:
p_max, p_min

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from utils import compute_pressForce, compute_orthonormal, linear_transform, update_data
        
optim_iter = 2000
is_objvis = True

fig, ax = plt.subplots(figsize=(4,4), ncols=1)
bd = testdata.param["n0"].reshape(40,2).detach().cpu().numpy()
bd = (((bd/2) + 0.5) * 62) + 0
#print(a.shape)
mappable0 = ax.plot(bd[:,0], bd[:,1])
ax.set_xlim(0, 62)
ax.set_ylim(0, 62)
plt.show()


fig, ax = plt.subplots(figsize=(4,4), ncols=1)
ax.imshow(testdata.node_feature["n0"][:,-1,-1].reshape(62,62).detach().cpu().numpy(),
          cmap='viridis',
          aspect='auto')
plt.show()

# fig, ax = plt.subplots(figsize=(4,4), ncols=1)
# ax.imshow(testdata.node_label["n0"][:,-1,-1].reshape(62,62).detach().cpu().numpy(),
#           cmap='viridis',
#           aspect='auto')
# plt.show()


fig, ax = plt.subplots(figsize=(8,8), ncols=1)
length, nx, ny, cen = compute_orthonormal(torch.tensor(bd))
rec_press = ((((testdata.node_label["n0"][:,-1,-1].reshape(62,62)/2) + 0.5) * (p_max-p_min)) + p_min).detach().cpu()
lin_press = linear_transform(torch.nn.functional.pad(rec_press, (1,3,1,3)), cen) #*-1
print(lin_press)
mappable0 = ax.plot(bd[:,0], bd[:,1])
normals = torch.stack((lin_press*nx,lin_press*ny), -1)
for i in range(40):
    rel_normals = cen[i,:] + normals[i,:]
    ax.plot((cen[i,0], rel_normals[0]), (cen[i,1], rel_normals[1]))
ax.set_xlim(24, 35)
ax.set_ylim(32, 43)
# ax.set_xlim(18, 28)
# ax.set_ylim(37, 47)
plt.show()

pdf = PdfPages('./optimized_naca_lepde_unflip.pdf')
list_force = []
list_drag_force = []
for oiter in range(optim_iter):
    total_x_force = 0
    total_y_force = 0

    bound = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:].reshape(40,2)

    rec_bound = (((bound/2) + 0.5) * 62) + 0
    up_solid_mask, up_solid_offset = update_static_masks(rec_bound)
    tor_batch_mask = torch.where(up_solid_mask==1, False, True).clone().flatten()
    up_solid_mask = up_solid_mask[...,None]
    static_feat = torch.cat((up_solid_mask, up_solid_offset), -1)
    mul_static_feat = torch.stack([static_feat for _ in range(4)], -2).reshape(-1,4,3)
    
    
    a = deepsnap_Batch
    batch, _ = a._init_batch_fields(testdata.keys, [])
    batch.batch = testdata.batch.clone()
    batch.compute_func = testdata.compute_func
    batch.directed = testdata.directed.detach().clone()
    batch.dyn_dims = testdata.dyn_dims
    batch.edge_attr = testdata.edge_attr
    batch.edge_index = {('n0','0','n0'): testdata.edge_index[('n0','0','n0')].detach().clone()}
    batch.edge_label_index = {('n0','0','n0'): testdata.edge_label_index[('n0','0','n0')].detach().clone()}
    batch.grid_keys = testdata.grid_keys
    batch.mask = {"n0": tor_batch_mask.detach()}
    batch.node_feature = {"n0": torch.cat((mul_static_feat, orgdata.node_feature["n0"][...,3:].detach()), -1)}
    batch.node_label = {"n0": testdata.node_label["n0"].detach().clone()}
    batch.node_label_index = {"n0": testdata.node_label_index["n0"].detach().clone()}
    batch.node_pos = {"n0": testdata.node_pos["n0"].detach().clone()}
    batch.original_shape = testdata.original_shape
    batch.param = {"n0": torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:]}
    batch.params = testdata.params
    batch.part_keys = testdata.part_keys
    batch.task = testdata.task
    
    testdata = batch
    
#     for _ in range(6):
#         testdata, pred = get_data_next_step(model, testdata, use_grads=False, return_data=True, is_y_diff=False)
#         press = ((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min
#         # press = ((((testdata.node_label["n0"][:,-1,-1].reshape(62,62)/2) + 0.5) * (p_max-p_min)) + p_min) #.detach().cpu()
#         raw_bound = (((testdata.param["n0"].reshape(40,2)/2) + 0.5) * 62) + 0
        
#         length, nx, ny, cen = compute_orthonormal(torch.tensor(raw_bound))
#         cen = cen.to(device)
#         lin_press = linear_transform(torch.nn.functional.pad(press, (1,3,1,3)), cen)
#         x_force, y_force = compute_pressForce(torch.nn.functional.pad(press, (1,3,1,3)), raw_bound)
#         total_x_force += x_force
#         total_y_force += y_force
    for kk in range(prerollout+one_period):
        if oiter % 50 == 49 and kk == 0 and vis_prerollout:
            print("kk = 0")
            fig, ax = plt.subplots(figsize=(4,4), ncols=1)
            ax.imshow(torch.nn.functional.pad(((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min, ((1,3,1,3))).detach().cpu().numpy(), cmap='viridis',
                     aspect='auto',
                     origin='lower')
            plt.show()
            # print(cen.shape, raw_bound.shape)

        testdata, pred = get_data_next_step(model, testdata, use_grads=False, return_data=True, is_y_diff=False)
        # pdb.set_trace()
        press = ((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min
        # press = ((((testdata.node_label["n0"][:,-1,-1].reshape(62,62)/2) + 0.5) * (p_max-p_min)) + p_min) #.detach().cpu()
        raw_bound = (((testdata.param["n0"].reshape(40,2)/2) + 0.5) * 62) + 0
        
        length, nx, ny, cen = compute_orthonormal(torch.tensor(raw_bound))
        cen = cen.to(device)
        
        if oiter % 50 == 49 and kk == prerollout and vis_prerollout:
            print("kk = " + str(prerollout))
            fig, ax = plt.subplots(figsize=(4,4), ncols=1)
            ax.imshow(torch.nn.functional.pad(press, ((1,3,1,3))).detach().cpu().numpy(), cmap='viridis',
                     aspect='auto',
                     origin='lower')
            plt.show()
            # print(cen.shape, raw_bound.shape)
        
        if kk >= prerollout:
            testdata, pred = get_data_next_step(model, testdata, use_grads=False, return_data=True, is_y_diff=False)
            # pdb.set_trace()
            press = ((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min
            # press = ((((testdata.node_label["n0"][:,-1,-1].reshape(62,62)/2) + 0.5) * (p_max-p_min)) + p_min) #.detach().cpu()
            raw_bound = (((testdata.param["n0"].reshape(40,2)/2) + 0.5) * 62) + 0
            lin_press = linear_transform(torch.nn.functional.pad(press, (1,3,1,3)), cen)
            x_force, y_force = compute_pressForce(torch.nn.functional.pad(press, (1,3,1,3)), raw_bound)
            
            total_x_force += x_force
            total_y_force += y_force
        
    total_x_force = total_x_force/one_period
    total_y_force = total_y_force/one_period
        
    list_force.append(total_y_force.item())
    list_drag_force.append(total_x_force.item())

    output = -1/total_x_force + 10 * torch.square(-30 - total_y_force)
    # output = -1/total_x_force # + 10 * torch.square(-30 - total_y_force)
    optimizer.zero_grad()
    output.backward()
    torch.nn.utils.clip_grad_value_(opt_variable, 0.01)
    # print("Gradient: ", opt_variable.grad)
    optimizer.step()
    aft_bd = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:].reshape(40,2).clone()

    ### Visualization of updated boundary """
    if oiter % 50 == 49:
    # if oiter % 10 == 9:
        print("iteration: ", oiter)
        bound = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:].reshape(40,2)
        bd = (((bound.detach().cpu().numpy()/2) + 0.5) * 62) + 0        
        length, nx, ny, cen = compute_orthonormal(torch.tensor(bd))

        fig, ax = plt.subplots(figsize=(18,3), ncols=6)
        mappable0 = ax[0].plot(bd[:,0], bd[:,1])
        nx = nx.detach().cpu()
        ny = ny.detach().cpu()
        cen = cen.detach().cpu()
        lin_press = lin_press.cpu()
        normals = torch.stack((lin_press*nx,lin_press*ny), -1)#.to(device)
        for i in range(40):
            rel_normals = cen[i,:] + normals[i,:]
            ax[0].plot((cen[i,0].numpy(), rel_normals[0].detach().numpy()), (cen[i,1].numpy(), rel_normals[1].detach().numpy()))
        ax[0].set_xlim(24, 35)
        ax[0].set_ylim(32, 43)
        mappable1 = ax[1].imshow(testdata.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')
        vis_offsetmask = torch.where(testdata.node_feature["n0"][:,-1,1]!=0, 1, 0)
        mappable2 = ax[2].imshow(vis_offsetmask.reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')        
        mappable3 = ax[3].imshow(testdata.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy()-vis_offsetmask.reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')        
        mappable4 = ax[4].plot(np.array(list_force)[0::5])
        mappable5 = ax[5].plot(np.array(list_drag_force)[0::5])
         # pdf.savefig()
        plt.show()
        
        print("diff_bef_aft ", (aft_bd - bound).sum())
    
pdf.close()

## 4. inverse optimization with FNO

In [ ]:
isplot = True
all_hash = [
    #"Yirzlp+j_ampere4",
    "clnAWVnz_hyperturing1",
    
]
hash_str = all_hash[0]
dirname = EXP_PATH + "naca_ellipse_2023-06-02/"
filename = filter_filename(dirname, include=hash_str)
if len(filename) == 0:
    raise

try:
    data_record = pload(dirname + filename[0])
except Exception as e:
    print(f"error {e}")
    # continue
    raise
if isplot:
    plot_learning_curve(data_record)
args = init_args(update_legacy_default_hyperparam(data_record["args"]))
args.filename = filename
# model = load_model(data_record["best_model_dict"], device=device)
model_fno = load_model(data_record["model_dict"][-1], device=device)
model_fno.eval()
p.print(filename, banner_size=100)

# Load test dataset:
args_test = deepcopy(args)
if args.temporal_bundle_steps == 1:
    if args.dataset in ["fno", "fno-2", "fno-3"]:
        args_test.multi_step = "20"
    elif args.dataset in ["fno-1"]:
        args_test.multi_step = "40"
    elif args.dataset in ["fno-4"]:
        args_test.multi_step = "10"
    elif args.dataset in ["naca_ellipse_lepde"]:
        args_test.multi_step = "1"
        args_test.latent_multi_step="1"
    else:
        raise
else:
    pass
args_test.batch_size = 1
args_test.is_test_only=True


In [ ]:
# Clone data
testdata = data.clone()
orgdata = data.clone()

# Set leaf variables for optimization
const_variable = testdata.param["n0"][:,0::2].detach().clone()
opt_variable = testdata.param["n0"][:,1::2].detach().clone()
opt_variable.requires_grad=True
testdata.param["n0"] = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:]

optimizer = torch.optim.Adam([opt_variable], lr=0.00001)

prerollout = 36
one_period = 4
vis_prerollout = False

optim_iter = 1300

In [ ]:
# Constants for normalization

normalization_filename = os.path.join("./dataset/naca_ellipse/training_trajectories/", "normalization_max_min.p")
normdict = pickle.load(open(normalization_filename, "rb"))
x_max = normdict["x_max"]
x_min = normdict["x_min"]
y_max = normdict["y_max"]
y_min = normdict["y_min"]
p_max = normdict["p_max"]
p_min = normdict["p_min"]
p_max = p_max.to(device)
p_min = p_min.to(device)

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
from utils import compute_pressForce, compute_orthonormal, linear_transform, update_data

fig, ax = plt.subplots(figsize=(4,4), ncols=1)
bd = testdata.param["n0"].reshape(40,2).detach().cpu().numpy()
bd = (((bd/2) + 0.5) * 62) + 0
#print(a.shape)
mappable0 = ax.plot(bd[:,0], bd[:,1])
ax.set_xlim(0, 62)
ax.set_ylim(0, 62)
plt.show()

fig, ax = plt.subplots(figsize=(4,4), ncols=1)
ax.imshow(testdata.node_feature["n0"][:,-1,-1].reshape(62,62).detach().cpu().numpy(), cmap='viridis',
         aspect='auto',
         origin='lower')
plt.show()

fig, ax = plt.subplots(figsize=(8,8), ncols=1)
length, nx, ny, cen = compute_orthonormal(torch.tensor(bd))
rec_press = ((((testdata.node_label["n0"][:,-1,-1].reshape(62,62)/2) + 0.5) * (p_max-p_min)) + p_min).detach().cpu()
lin_press = linear_transform(torch.nn.functional.pad(rec_press, (1,3,1,3)), cen) #* (-1)
print(lin_press)
mappable0 = ax.plot(bd[:,0], bd[:,1])
normals = torch.stack((lin_press*nx,lin_press*ny), -1)
for i in range(40):
    rel_normals = cen[i,:] + normals[i,:]
    ax.plot((cen[i,0], rel_normals[0]), (cen[i,1], rel_normals[1]))
ax.set_xlim(24, 35)
ax.set_ylim(32, 43)
# ax.set_xlim(18, 28)
# ax.set_ylim(37, 47)
plt.show()

In [ ]:
pdf = PdfPages('./optimized_naca_fno_gradient.pdf')

list_force = []
list_drag_force = []
for oiter in range(optim_iter):
    total_x_force = 0
    total_y_force = 0

    ### Define boundary ###
    bound = torch.cat((const_variable, opt_variable), 0).transpose(1,0).flatten()[None,:].reshape(40,2)

    
    ### update boundary ###
    rec_bound = (((bound/2) + 0.5) * 62) + 0  
    testdata = update_data(rec_bound, testdata, orgdata, const_variable, opt_variable)
    raw_bound = (((testdata.param["n0"].reshape(40,2)/2) + 0.5) * 62) + 0

    
    ### Perform rollout and Compute objective ###
    for kk in range(prerollout+one_period):
        if oiter % 50 == 49 and kk == 0 and vis_prerollout:
            print("kk = 0")
            fig, ax = plt.subplots(figsize=(4,4), ncols=1)
            ax.imshow(torch.nn.functional.pad(((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min, ((1,3,1,3))).detach().cpu().numpy(), cmap='viridis',
                     aspect='auto',
                     origin='lower')
            plt.show()

        testdata, pred = get_data_next_step(model_fno, testdata, use_grads=False, return_data=True, is_y_diff=False)
        press = ((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min
        
        if oiter % 50 == 49 and kk == prerollout and vis_prerollout:
            print("kk = " + str(prerollout))
            fig, ax = plt.subplots(figsize=(4,4), ncols=1)
            ax.imshow(torch.nn.functional.pad(press, ((1,3,1,3))).detach().cpu().numpy(), cmap='viridis',
                     aspect='auto',
                     origin='lower')
            plt.show()
        
        if kk >= prerollout:
            testdata, pred = get_data_next_step(model_fno, testdata, use_grads=False, return_data=True, is_y_diff=False)
            press = ((((pred["n0"].reshape(62, 62, 1, 3)[...,0,-1])/2) + 0.5) * (p_max-p_min)) + p_min
            x_force, y_force = compute_pressForce(torch.nn.functional.pad(press, (1,3,1,3)), raw_bound)
            
            total_x_force += x_force
            total_y_force += y_force

    total_x_force = total_x_force/one_period
    total_y_force = total_y_force/one_period

    list_force.append(total_y_force.item())
    list_drag_force.append(total_x_force.item())

    
    ### Perform optimization ###
    # output = -1/total_x_force + 5 * torch.square(-7 - total_y_force)
    output = total_x_force 
    optimizer.zero_grad()
    output.backward()
    torch.nn.utils.clip_grad_value_(opt_variable, 0.01)
    optimizer.step()

    rolled_boundary = torch.roll(bound, -1, 0)
    bd_diff = torch.abs(bound - rolled_boundary)
    if (bd_diff > 2).sum() > 0:
        import pdb
        pdb.set_trace()

        
    ### Visualization ###
    if oiter % 50 == 49:
        print("iteration: ", oiter + 1)
        print("objective: ", output)

        bd = (((bound.detach().cpu().numpy()/2) + 0.5) * 62) + 0
        length, nx, ny, cen = compute_orthonormal(torch.tensor(bd))
        cen = cen.to(device)
        lin_press = linear_transform(torch.nn.functional.pad(press, (1,3,1,3)), cen)

        fig, ax = plt.subplots(figsize=(18,3), ncols=6)
        mappable0 = ax[0].plot(bd[:,0], bd[:,1])
        nx = nx.detach().cpu()
        ny = ny.detach().cpu()
        cen = cen.detach().cpu()
        lin_press = lin_press.cpu()
        # print(cen.device, normals.device, lin_press.device)
        normals = torch.stack((lin_press*nx,lin_press*ny), -1)
        for i in range(40):
            rel_normals = cen[i,:] + normals[i,:]
            ax[0].plot((cen[i,0].numpy(), rel_normals[0].detach().numpy()), (cen[i,1].numpy(), rel_normals[1].detach().numpy()))
        ax[0].set_xlim(24, 35)
        ax[0].set_ylim(32, 43)
        mappable1 = ax[1].imshow(testdata.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')
        vis_offsetmask = torch.where(testdata.node_feature["n0"][:,-1,1]!=0, 1, 0)
        mappable2 = ax[2].imshow(vis_offsetmask.reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')        
        mappable3 = ax[3].imshow(testdata.node_feature["n0"][:,-1,0].reshape(62,62).detach().cpu().numpy()-vis_offsetmask.reshape(62,62).detach().cpu().numpy(), cmap='viridis',
                                 aspect='auto',
                                 origin='lower')        
        mappable4 = ax[4].plot(np.array(list_force)[0::5])
        mappable5 = ax[5].plot(np.array(list_drag_force)[0::5])
        pdf.savefig()
        plt.show()
    
pdf.close()